In [1]:
wd = '/mnt/openfact/users/msawinski/factue-task2'
wd = '/Users/marcinsawinski/Documents/GitHub/factue-task2'
import sys, os
os.chdir(wd)

In [12]:
import pandas as pd
from pathlib import Path
import os
root = Path("data/llm_output/persuasion")
# Example path template
files = root.rglob("*/*.parquet")

# Read all files and add file path as a column
df_list = []
for f in files:
    df_part = pd.read_parquet(f)
    df_part['source_file'] = f  # add the file path
    df_list.append(df_part)

# Combine into one DataFrame
df = pd.concat(df_list, ignore_index=True)
df['split'] = df.source_file.astype(str).str.split('/',expand=True)[7]

In [7]:
df.head(3)

,filename,start,end,text_lang,text,label_bin,labels_multi,output_id,job,step,...,Category,Label,Conclusion,Technique,Justification,Evidence,ValueAppealedTo,Explanation,Appeal to Fear – Prejudice score,Result
0,RU_discrediting_the_west_1.txt,0,26,RU,Проект: «Расчеловечивание»,False,[],LLAMA_31_8B/attack/Questioning_the_Reputation_...,persuasion,detect,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,RU_discrediting_the_west_1.txt,28,207,RU,США и страны Запада всегда были одержимы идеей...,True,[Questioning_the_Reputation],LLAMA_31_8B/attack/Questioning_the_Reputation_...,persuasion,detect,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,RU_discrediting_the_west_1.txt,209,294,RU,Но мир меняется. Другие государства развиваютс...,False,[],LLAMA_31_8B/attack/Questioning_the_Reputation_...,persuasion,detect,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
df[df.split=='train'].groupby(["technique_id","text_lang","model_name","split"])['filename'].agg("count").reset_index()

,technique_id,text_lang,model_name,split,filename
0,Appeal_to_Fear-Prejudice,BG,gpt-4o-mini,train,363
1,Appeal_to_Fear-Prejudice,BG,llama3.1:8b,train,140
2,Appeal_to_Fear-Prejudice,PL,gpt-4o-mini,train,289
3,Appeal_to_Fear-Prejudice,PL,llama3.1:8b,train,100
4,Appeal_to_Fear-Prejudice,RU,gpt-4o-mini,train,239
5,Appeal_to_Fear-Prejudice,RU,llama3.1:8b,train,60
6,Appeal_to_Fear-Prejudice,SI,gpt-4o-mini,train,108
7,Appeal_to_Fear-Prejudice,SI,llama3.1:8b,train,40
8,Appeal_to_Values,BG,gpt-4o-mini,train,363
9,Appeal_to_Values,BG,llama3.1:8b,train,363


In [3]:
def normalize_binary(x):
    return 1 if str(x).strip().lower() in {'1', 'true'} else 0

In [4]:
df['source_file'].value_counts().sort_index()
df['gold'] = df['gold'].apply(normalize_binary)
df['pred'] = df['pred'].apply(normalize_binary)
df['split'] = df.source_file.astype(str).str.split('/',expand=True)[7]

In [5]:
df.technique_id.value_counts()

technique_id
Appeal_to_Values              2256
Questioning_the_Reputation    2236
Loaded_Language               1877
Appeal_to_Fear-Prejudice      1597
Name_Calling-Labeling         1257
Doubt                         1257
Exaggeration-Minimisation     1257
Repetition                     258
Name: count, dtype: int64

In [6]:
df

,filename,start,end,text_lang,text,label_bin,labels_multi,output_id,job,step,...,Label,Conclusion,Technique,Justification,Evidence,ValueAppealedTo,Explanation,Appeal to Fear – Prejudice score,Result,split
0,RU_discrediting_the_west_1.txt,0,26,RU,Проект: «Расчеловечивание»,False,[],LLAMA_31_8B/attack/Questioning_the_Reputation_...,persuasion,detect,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,trial
1,RU_discrediting_the_west_1.txt,28,207,RU,США и страны Запада всегда были одержимы идеей...,True,[Questioning_the_Reputation],LLAMA_31_8B/attack/Questioning_the_Reputation_...,persuasion,detect,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,trial
2,RU_discrediting_the_west_1.txt,209,294,RU,Но мир меняется. Другие государства развиваютс...,False,[],LLAMA_31_8B/attack/Questioning_the_Reputation_...,persuasion,detect,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,trial
3,RU_discrediting_the_west_1.txt,296,416,RU,Вашингтон и Брюссель теряют своё доминирование...,True,[Questioning_the_Reputation],LLAMA_31_8B/attack/Questioning_the_Reputation_...,persuasion,detect,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,trial
4,RU_discrediting_the_west_1.txt,418,467,RU,Цель которого — чтобы люди перестали быть людьми.,False,[],LLAMA_31_8B/attack/Questioning_the_Reputation_...,persuasion,detect,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,trial
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11990,pl_eu_12_06_2024_n01.txt,3549,3899,PL,Wicemarszałek Włodzimierz Czarzasty:\nDziękuję...,False,[],GPT_4O_MINI/manipulative/Exaggeration-Minimisa...,persuasion,detect,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,train
11991,pl_eu_12_06_2024_n01.txt,3901,4528,PL,Poseł Szymon Szynkowski vel Sęk:\nPanie Marsza...,False,[],GPT_4O_MINI/manipulative/Exaggeration-Minimisa...,persuasion,detect,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,train
11992,pl_eu_12_06_2024_n01.txt,4530,5548,PL,"Natomiast to, co jest kłopotem przedłożonego p...",True,[Doubt],GPT_4O_MINI/manipulative/Exaggeration-Minimisa...,persuasion,detect,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,train
11993,pl_eu_12_06_2024_n01.txt,5550,6082,PL,"W art. 15 ust. 2 projektu przewidziano, że zam...",True,[Doubt],GPT_4O_MINI/manipulative/Exaggeration-Minimisa...,persuasion,detect,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,train


In [7]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
# Define a function to compute metrics for a group
# Assume df has columns: technique_id, text_lang, gold, pred
rows = []

def is_valid_label(x):
    return x in (0, 1)

rows = []

for technique_id in df['technique_id'].unique():
    for text_lang in df.loc[df['technique_id'] == technique_id, 'text_lang'].unique():
        for model_name in df.loc[(df['technique_id'] == technique_id) & (df['text_lang'] == text_lang), 'model_name'].unique():
            group = df[
                (df['technique_id'] == technique_id) &
                (df['text_lang'] == text_lang) &
                (df['model_name'] == model_name)
            ]
            if len(group) > 0:
                row = {
                    'technique_id': technique_id,
                    'text_lang': text_lang,
                    'model_name': model_name,
                    'accuracy': accuracy_score(group['gold'], group['pred']),
                    'precision': precision_score(group['gold'], group['pred'], zero_division=0),
                    'recall': recall_score(group['gold'], group['pred'], zero_division=0),
                    'f1': f1_score(group['gold'], group['pred'], zero_division=0),
                    'support': len(group)
                }
                rows.append(row)

results = pd.DataFrame(rows)
results[['accuracy', 'precision','recall', 'f1']] = results[['accuracy', 'precision','recall', 'f1']].round(2)
results

,technique_id,text_lang,model_name,accuracy,precision,recall,f1,support
0,Questioning_the_Reputation,RU,llama3.1:8b,0.56,0.06,1.00,0.11,257
1,Questioning_the_Reputation,RU,gpt-4o-mini,0.70,0.08,1.00,0.16,257
2,Questioning_the_Reputation,PL,llama3.1:8b,0.79,0.43,0.72,0.54,296
3,Questioning_the_Reputation,PL,gpt-4o-mini,0.85,0.57,0.48,0.52,316
4,Questioning_the_Reputation,BG,llama3.1:8b,0.73,0.33,0.80,0.47,438
...,...,...,...,...,...,...,...,...
59,Exaggeration-Minimisation,PL,gpt-4o-mini,0.56,0.12,0.90,0.21,316
60,Exaggeration-Minimisation,BG,llama3.1:8b,0.65,0.00,0.00,0.00,75
61,Exaggeration-Minimisation,BG,gpt-4o-mini,0.69,0.21,0.95,0.35,438
62,Exaggeration-Minimisation,SI,llama3.1:8b,0.78,0.00,0.00,0.00,9


In [8]:
import seaborn as sns
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'seaborn'

In [ ]:
fig, axes = plt.subplots(len(metrics), 1, figsize=(10, 18), sharex=True)

for i, metric in enumerate(metrics):
    ax = axes[i]
    sns.barplot(
        data=results,
        y='technique_id',
        x=metric,
        hue='text_lang',
        ax=ax,
        palette='muted',
        errorbar=None  # fallback for compatibility
    )
    ax.set_title(metric.capitalize())
    ax.set_xlim(0, 1.05)
    ax.set_ylabel('Technique ID')
    ax.set_xlabel('Score')
    ax.legend(title='Text Language', loc='lower right')

plt.tight_layout()
plt.show()

In [ ]:
# # Create a melt of metrics for faceted visualization
# metrics_df = results.melt(
#     id_vars=['technique_id', 'text_lang'],
#     value_vars=['accuracy', 'precision', 'recall', 'f1'],
#     var_name='metric',
#     value_name='score'
# )

# # Plot
# plt.figure(figsize=(12, 6))
# sns.barplot(data=metrics_df, x='technique_id', y='score', hue='text_lang', palette='muted', errorbar=None)
# plt.title('Metrics by Technique and Language')
# plt.ylim(0, 1.05)
# plt.ylabel('Score')
# plt.xlabel('Technique ID')
# plt.legend(title='Text Language')
# plt.grid(axis='y', linestyle='--', alpha=0.7)
# plt.tight_layout()
# plt.show()

# total

In [ ]:
# Metrics
acc = accuracy_score(df['gold'], df['pred'])
precision = precision_score(df['gold'], df['pred'], zero_division=0)
recall = recall_score(df['gold'], df['pred'], average='binary')
f1 = f1_score(df['gold'], df['pred'], average='binary')

print(f"Accuracy: {acc:.3f}")
print(f"Precision: {precision:.3f}")
print(f"Recall: {recall:.3f}")
print(f"F1-score: {f1:.3f}")